<a href="https://colab.research.google.com/github/vncecruz/Emptech-2/blob/main/Cruz_Assignment_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Submission Details

Name: Vince Joseph B. Cruz<br>
Course and Section: CPE 019-CPE32S1 - Emerging Technologies 2 in CpE <br>
Date of Submission: June 23, 2024 <br>
Instructor: Engr. Roman Richard<br>

---

# Part 1: Try the MLP Notebook using the CIFAR10 Keras Dataset

# Part 2:


*  Choose any dataset

*   Explain the problem you are trying to solve

*   Create your own model

*   Evaluate the accuracy of your model

In [69]:
from google.colab import drive

**Explain the problem you are trying to solve**

In [70]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

ytch = pd.read_csv("/content/drive/MyDrive/Emptech/youtube.csv")

In [71]:
ytch.info()
ytch.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   RANK                    500 non-null    object
 1   NAME_OF_CHANNEL         493 non-null    object
 2   TOTAL_NUMBER_OF_VIDEOS  500 non-null    object
 3   SUBSCRIBERS             500 non-null    object
 4   VIEWS                   500 non-null    object
 5   CATEGORY                492 non-null    object
dtypes: object(6)
memory usage: 23.6+ KB


,RANK,NAME_OF_CHANNEL,TOTAL_NUMBER_OF_VIDEOS,SUBSCRIBERS,VIEWS,CATEGORY
count,500,493,500,500,500,492
unique,500,493,444,246,471,32
top,#1,MrBeast,1.07K,20.9M,6.01B,Entertainment
freq,1,1,5,10,3,186


In [72]:
ytch.head()

,RANK,NAME_OF_CHANNEL,TOTAL_NUMBER_OF_VIDEOS,SUBSCRIBERS,VIEWS,CATEGORY
0,#1,MrBeast,799,274M,50.98B,Entertainment
1,#2,T-Series,21.12K,267M,257.16B,Music
2,#3,Cocomelon - Nursery Rhymes,1.18K,176M,182.88B,Kids
3,#4,SET India,138.97K,173M,164.71B,Entertainment
4,#5,✿ Kids Diana Show,1.22K,123M,103.5B,Kids


In [73]:
ytch.tail()

,RANK,NAME_OF_CHANNEL,TOTAL_NUMBER_OF_VIDEOS,SUBSCRIBERS,VIEWS,CATEGORY
495,#496,Smile Family Spanish,563,20.5M,7.63B,Entertainment
496,#497,Marvel Entertainment,8.97K,20.5M,6.01B,Entertainment
497,#498,5-Minute Crafts DIY,6.1K,20.5M,5.16B,NaN
498,#499,SatoshiSea,4,20.5M,1.55M,NaN
499,#500,Britain's Got Talent,4.63K,20.4M,11.76B,NaN


**Explain the problem you are trying to solve**

The problem that is trying to be solved is to predict subscriber counts for YouTube channels based on features like total number of videos, views, and category. This predictive model aims to provide insights into channel popularity and audience engagement dynamics on the YouTube platform.

**Create your own model**

In [227]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import tensorflow as tf
from scipy.sparse import csr_matrix, hstack

def convert_to_numeric(value):
    if isinstance(value, str):
        if 'K' in value:
            return float(value.replace('K', '')) * 1e3
        elif 'M' in value:
            return float(value.replace('M', '')) * 1e6
        elif 'B' in value:
            return float(value.replace('B', '')) * 1e9
    return float(value)

In [189]:
ytch['TOTAL_NUMBER_OF_VIDEOS'] = ytch['TOTAL_NUMBER_OF_VIDEOS'].apply(convert_to_numeric)
ytch['VIEWS'] = ytch['VIEWS'].apply(convert_to_numeric)
ytch['SUBSCRIBERS'] = ytch['SUBSCRIBERS'].apply(convert_to_numeric)

In [190]:
X = ytch.drop(columns=['RANK', 'NAME_OF_CHANNEL', 'SUBSCRIBERS'])
y = ytch['SUBSCRIBERS']

In [191]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [192]:
numerical_features = ['TOTAL_NUMBER_OF_VIDEOS', 'VIEWS']
categorical_features = ['CATEGORY']

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [217]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

X_train_transformed = pipeline.fit_transform(X_train)

X_test_transformed = pipeline.transform(X_test)

categorical_names = preprocessor.named_transformers_['cat'].categories_[0]
feature_names = numerical_features + list(categorical_names)

X_train_df = pd.DataFrame(X_train_transformed.toarray(), columns=feature_names)
X_test_df = pd.DataFrame(X_test_transformed.toarray(), columns=feature_names)

In [210]:
X_train_df = pd.DataFrame(X_train, columns=feature_names)

In [211]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, input_dim=X_train_df.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

history = model.fit(X_train_df, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
10/10 [==============================] - 1s 36ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/50
10/10 [==============================] - 0s 7ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/50
10/10 [==============================] - 0s 7ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/50
10/10 [==============================] - 0s 7ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/50
10/10 [==============================] - 0s 8ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/50
10/10 [==============================] - 0s 8ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/50
10/10 [==============================] - 0s 7ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/50
10/10 [==============================] - 0s 7ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/50
10/10 [==============================] - 0s 

**Evaluate the accuracy of your model**

In [232]:
X_test_array = X_test_df.to_numpy()
score = model.evaluate(X_test_array, y_test)
print("Test MSE:", score[0])
print("Test MAE:", score[1])

4/4 [==============================] - 0s 5ms/step - loss: nan - mae: nan
Test MSE: nan
Test MAE: nan


# Conclusion

In this activity
we used and choose our own dataset it was difficult because there is more guide but I used the previous codes and activities and also the one in the modules but I still couldn't solve the error. The errors that I have encountered is 'csr_matrix' object has no attribute 'columns' and ValueError: Shape of passed values is (400, 1), indices imply (400, 28). I thought by adding this code from scipy.sparse import csr_matrix, hstack it will fix the error.


